# IMDB sentiment analysis

(Based on Coursera MIPT & Yandex Machine Learning course)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

There are 25000 IMDB movie reviews, specially selected for sentiment analysis. The sentiment of reviews is binary, meaning the IMDB rating < 5 results in a sentiment score of 0, and rating >=7 have a sentiment score of 1.

Kaggle competition: https://www.kaggle.com/c/word2vec-nlp-tutorial/data

Let's load the sample:

In [6]:
imdb = pd.read_csv('labeledTrainData.tsv', delimiter='\t')
imdb.shape

(25000, 3)

In [7]:
imdb.head()

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


Classes are balanced:

In [8]:
imdb.sentiment.value_counts()

1    12500
0    12500
Name: sentiment, dtype: int64

Splitting the training and test sets:

In [9]:
from sklearn.cross_validation import train_test_split
texts_train, texts_test, y_train, y_test = train_test_split(imdb.review.values, imdb.sentiment.values)

Vectorizing reviews:

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

vect = TfidfVectorizer(sublinear_tf=True, use_idf=True)
X_train = vect.fit_transform(texts_train)
X_test = vect.transform(texts_test)

## Logistic regression

Let's fit Logistic Regression and evaluate AUC score:

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

clf = LogisticRegression()
clf.fit(X_train, y_train)

print metrics.accuracy_score(y_test, clf.predict(X_test))
print metrics.roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])

0.88528
0.957477964574


But we've got too many features:

In [14]:
X_train.shape

(18750, 66542)

Let's perform selection using Lasso regularization:

In [12]:
clf = LogisticRegression(C=0.15, penalty='l1')
clf.fit(X_train, y_train)

print np.sum(np.abs(clf.coef_) > 1e-4)
print metrics.accuracy_score(y_test, clf.predict(X_test))
print metrics.roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])

104
0.81856
0.896653687334


Another approach for a feature selection - randomized logistic regression (subsampling + l1):

In [ ]:
from sklearn.linear_model import RandomizedLogisticRegression

rlg = RandomizedLogisticRegression(C=0.15)
rlg.fit(X_train, y_train)
#careful! throws warnings

Let's see how many features selected:

In [21]:
np.sum(rlg.scores_ > 0)

140

Fitting Logistic Regression on preselected 140 features:

In [22]:
X_train_lasso = X_train[:, rlg.scores_ > 0]
X_test_lasso = X_test[:, rlg.scores_ > 0]

In [23]:
clf = LogisticRegression(C=1)
clf.fit(X_train_lasso, y_train)
print metrics.accuracy_score(y_test, clf.predict(X_test_lasso))
print metrics.roc_auc_score(y_test, clf.predict_proba(X_test_lasso)[:, 1])

0.83456
0.917981777733


## Principal component analysis approach

Let's create 100 synthetic features using PCA method:

In [24]:
from sklearn.decomposition import TruncatedSVD

tsvd = TruncatedSVD(n_components=100)
X_train_pca = tsvd.fit_transform(X_train)
X_test_pca = tsvd.transform(X_test)

Fitting logistic regression on them:

In [25]:
clf = LogisticRegression()
clf.fit(X_train_pca, y_train)
print metrics.accuracy_score(y_test, clf.predict(X_test_pca))
print metrics.roc_auc_score(y_test, clf.predict_proba(X_test_pca)[:, 1])

0.85824
0.938035077479


On 100 features we get almost the same score as on whole 66k set. 

What about Random Forest?

In [27]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train_pca, y_train)
print metrics.accuracy_score(y_test, clf.predict(X_test_pca))
print metrics.roc_auc_score(y_test, clf.predict_proba(X_test_pca)[:, 1])

0.8312
0.907933367638


PCA features are optimal for linear methods, that's why logistic regression scores better then non-linear complex classifiers.